## Bayesian Analysis of 2020 Bundesliga Results

In [ ]:
library("rstan")
library("parallel")

: 

In [ ]:
example(stan_model, package = "rstan", run.dontrun = TRUE)

In [ ]:
options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)
Sys.setenv(LOCAL_CPPFLAGS = '-march=native')

In [ ]:
getwd()

In [ ]:
setwd("/mnt/d/Github/Stan/Stan")
remove(list = ls()) # clear the current work space

In [ ]:
HFA <- readRDS(file = paste(getwd(),
    "/Data/Sorted_Bundesliga_2019_20.rds", sep = ""))

In [ ]:
HFA

In [ ]:
ngames <- nrow(HFA)
ngames

nteams <- length(unique(HFA$host))
nteams

HFA$COVID + 1

dl <- list(
  ng = ngames,
  hg = HFA$HG,
  vg = HFA$VG,
  cv = HFA$COVID + 1,
  ht = HFA$host,
  vt = HFA$visitor,
  nt = nteams
)

In [ ]:
remove(sm)
sm <- stan_model(paste(getwd(), "/Models/Bundesliga2020.stan", sep = ""))

In [ ]:
fit <- stan(
  file = paste(getwd(), "/Models/Bundesliga2020.stan", sep = ""),  # Stan program
  data = dl,    # named list of data
  chains = 4,             # number of Markov chains
  warmup = 1000,          # number of warmup iterations per chain
  iter = 2000,            # total number of iterations per chain
  cores = 2,              # number of cores (could use one per chain)
  refresh = 1             # no progress shown
  )

In [ ]:
remove(fit)
fit <- sampling(object = sm,
                data = dl,
                init = "random",
                control = list(adapt_delta = 0.95),
                chains = 4,
                iter = 2000,
                warmup = 1500,
                thin = 1,
                verbose = TRUE)

In [ ]:
summary(fit, pars = c("hfa_d"))

In [ ]:
traceplot(fit, pars=c("hfa_d"))

In [ ]:
theme_posterior <- theme(
    axis.line.x = element_line(arrow=arrow(length=unit(0.05, "cm")),
     lineend = "butt"),
    panel.background = element_rect(fill="transparent"),
    panel.border = element_rect(fill="transparent"),
    plot.background = element_rect(fill = "transparent"),
  #panel.spacing.y = unit(1,"lines"),
    plot.margin = unit(c(1,1,1,1), "cm"),
  #
    axis.title.y = element_blank(), 
    axis.text.y = element_text(color = "grey20",
      size = 8, angle = 0, hjust = 1, vjust = 0, face = "plain"),
    xis.text.x = element_text(color="grey20",
      hjust = 1, vjust=1, size=8, angle = 50, face = "plain"),
    axis.ticks.y = element_blank()
)

In [ ]:
plot(fit,
     pars = names(fit)[c(1,2)],
     show_density = TRUE,
     fill_color = "#998811",
     est_color = "#ffffff",
     ci_level = 0.9, outer_level=0.95) +
     geom_vline(xintercept = 0, linetype = 3, size = 0.5)
     + theme_Posterior